#  Recomendador Generativo de Estrategias Comerciales (Seller Size + Performance)

Este notebook muestra cómo, a partir de la clasificación de cada seller en:
- `seller_size`
- `performance_level`

se construye un componente basado en GenAI que sugiere estrategias comerciales
personalizadas para el equipo comercial.

In [1]:
import pandas as pd
from pathlib import Path

# Ruta relativa desde el notebook
ROOT = Path("..").resolve()
PROFILE_PATH = ROOT / "data" / "processed" / "seller_profile.csv"

df = pd.read_csv(PROFILE_PATH)

df.head()

,seller_nickname,n_items,total_stock,logistic_type,total_value,avg_stock_per_item,n_categories,main_category,pct_main_category,pct_new,...,seller_reputation_score,seller_size,clasificacion_diversificacion,clasificacion_calidad,div_score,qual_score,log_score,total_score,performance_level,performance_segment
0,000631669c,1,10.0,Otro,7990.0,10.0,1,OTHER,1.0,1.0,...,0,Local Hero,Superficial,standard,1,1,1,3,Expected performance,Local Hero - Expected performance
1,0007153bca,2,55.0,XD,21945.0,27.5,1,OTHER,1.0,1.0,...,4,Core Seller,Especialista,confiable_gold,2,2,2,6,Diamante,Core Seller - Diamante
2,000bee3c3b,2,0.0,XD,0.0,0.0,1,"BOOKS, MULTIMEDIA & OTHER E!",1.0,1.0,...,0,Long Tail,Especialista,standard,2,1,2,5,Top performance,Long Tail - Top performance
3,000df2bd02,1,5.0,XD,7750.0,5.0,1,ACC CARS & VANS,1.0,0.0,...,4,Local Hero,Superficial,standard,1,1,2,4,Expected performance,Local Hero - Expected performance
4,000e27cea2,2,6.0,FBM,3070.0,3.0,2,HOUSEHOLD ITEMS,0.5,1.0,...,3,Long Tail,Híbrido,confiable_gold,2,2,0,4,Expected performance,Long Tail - Expected performance


In [2]:
df.columns

Index(['seller_nickname', 'n_items', 'total_stock', 'logistic_type',
       'total_value', 'avg_stock_per_item', 'n_categories', 'main_category',
       'pct_main_category', 'pct_new', 'pct_used', 'pct_refurbished',
       'avg_price_regular', 'median_price_regular', 'seller_reputation',
       'seller_reputation_score', 'seller_size',
       'clasificacion_diversificacion', 'clasificacion_calidad', 'div_score',
       'qual_score', 'log_score', 'total_score', 'performance_level',
       'performance_segment'],
      dtype='object')

## 1. Perfil mínimo utilizado para GenAI

Para este ejercicio solo utilizamos:
- `seller_nickname` (uuid)
- `seller_size`
- `performance_level`

In [3]:
profile_cols = [
    "seller_nickname",
    "seller_size",
    "performance_level",
]

seller_profile = df[profile_cols].copy()
seller_profile.head()

,seller_nickname,seller_size,performance_level
0,000631669c,Local Hero,Expected performance
1,0007153bca,Core Seller,Diamante
2,000bee3c3b,Long Tail,Top performance
3,000df2bd02,Local Hero,Expected performance
4,000e27cea2,Long Tail,Expected performance


### 2. Playbook por segmento

Definimos reglas base por combinación `(seller_size, performance_level)`.
Este playbook actúa como “guía” que el modelo generativo usará como contexto.

Construimos ese PLAYBOOK con Ayuda de la IA

In [5]:
PLAYBOOK = {
    ("Key Account", "Diamante"): {
        "objetivo": "Consolidar y expandir la relación con un socio estratégico clave.",
        "lineas": [
            "Definir un plan anual de co-marketing con foco en eventos pico.",
            "Negociar beneficios exclusivos (cuotas, bundles, lanzamientos anticipados).",
            "Explorar oportunidades de exclusividad en categorías clave.",
        ],
    },
    ("Key Account", "Top performance"): {
        "objetivo": "Mantener el alto rendimiento y capturar upside adicional.",
        "lineas": [
            "Refinar la calendarización de campañas comerciales y eventos.",
            "Profundizar surtido en SKUs de mayor conversión.",
            "Optimizar inversión en visibilidad según rentabilidad por categoría.",
        ],
    },
    ("Key Account", "Expected performance"): {
        "objetivo": "Cerrar brechas de ejecución para llevar al seller a un nivel superior.",
        "lineas": [
            "Revisar inventarios y disponibilidad en momentos pico.",
            "Alinear estrategia de precios y promociones con el mercado.",
            "Definir un roadmap de mejoras conjuntas en catálogo y contenido.",
        ],
    },
    ("Key Account", "Low performance"): {
        "objetivo": "Reducir riesgo y decidir si se reactiva o se desprioriza al seller.",
        "lineas": [
            "Realizar un diagnóstico claro de causas de bajo desempeño.",
            "Acordar un plan mínimo de remediación con hitos y plazos.",
            "Revisar si se mantiene, reduce o elimina inversión comercial.",
        ],
    },
    ("Core Seller", "Top performance"): {
        "objetivo": "Acelerar el crecimiento de un seller estable con buen potencial.",
        "lineas": [
            "Identificar categorías donde pueda escalar presencia.",
            "Explorar acciones de co-marketing a escala media.",
            "Definir incentivos por crecimiento de volumen y calidad.",
        ],
    },
    ("Local Hero", "Top performance"): {
        "objetivo": "Escalar un nicho rentable y fortalecer su liderazgo local.",
        "lineas": [
            "Aumentar visibilidad en su nicho principal.",
            "Expandir catálogo alrededor de sus best sellers.",
            "Explorar cross-sell hacia categorías afines.",
        ],
    },
    ("Long Tail", "Top performance"): {
        "objetivo": "Convertir un seller pequeño pero sólido en un 'Emerging Star'.",
        "lineas": [
            "Acompañar la expansión de catálogo de forma controlada.",
            "Mejorar contenido para aumentar conversión.",
            "Testear campañas de bajo costo para validar potencial.",
        ],
    },
    ("Long Tail", "Low performance"): {
        "objetivo": "Minimizar esfuerzo operativo en sellers con bajo impacto.",
        "lineas": [
            "Ofrecer recomendaciones básicas y auto-servicio.",
            "Limitar inversión manual del equipo comercial.",
            "Monitorear; solo re-escalar si se observa mejora clara.",
        ],
    },
}

## 3. Construcción del prompt

A partir de una fila con `seller_size` y `performance_level`, más el playbook,
generamos un prompt estructurado que será la entrada del modelo generativo.

In [6]:
def build_prompt_for_seller(row: pd.Series) -> str:
    nickname = row["seller_nickname"]
    size = row["seller_size"]
    level = row["performance_level"]

    key = (size, level)
    base = PLAYBOOK.get(
        key,
        {
            "objetivo": "Definir una estrategia comercial básica acorde al tamaño y nivel de performance del seller.",
            "lineas": [
                "Revisar catálogo y ajustar oferta a la demanda.",
                "Optimizar precios y promociones según su contexto competitivo.",
                "Definir acciones mínimas de mejora en servicio/logística.",
            ],
        },
    )

    prompt = f"""
Eres un analista comercial de Mercado Libre.

Perfil del seller:
- seller_nickname: {nickname}
- seller_size: {size}
- performance_level: {level}

Playbook de referencia para este segmento:
- Objetivo sugerido: {base["objetivo"]}
- Líneas sugeridas: {", ".join(base["lineas"])}

Con esta información, genera una estrategia comercial personalizada con el siguiente formato:

1) Objetivo principal (1 párrafo).
2) 3–5 acciones concretas para el equipo comercial, separadas por viñetas.
3) 2–3 KPIs clave para evaluar el impacto de la estrategia.

La respuesta debe ser clara, accionable y escrita en un lenguaje orientado a negocio.
"""
    return prompt

In [39]:
ejemplo = seller_profile.iloc[2]
print(build_prompt_for_seller(ejemplo))


Eres un analista comercial de Mercado Libre.

Perfil del seller:
- seller_nickname: 000bee3c3b
- seller_size: Long Tail
- performance_level: Top performance

Playbook de referencia para este segmento:
- Objetivo sugerido: Convertir un seller pequeño pero sólido en un 'Emerging Star'.
- Líneas sugeridas: Acompañar la expansión de catálogo de forma controlada., Mejorar contenido para aumentar conversión., Testear campañas de bajo costo para validar potencial.

Con esta información, genera una estrategia comercial personalizada con el siguiente formato:

1) Objetivo principal (1 párrafo).
2) 3–5 acciones concretas para el equipo comercial, separadas por viñetas.
3) 2–3 KPIs clave para evaluar el impacto de la estrategia.

La respuesta debe ser clara, accionable y escrita en un lenguaje orientado a negocio.



In [35]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

# Cliente global (usa OPENAI_API_KEY del entorno)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [47]:
def generate_strategy(row: pd.Series) -> str:
    """
    Genera una estrategia comercial usando la API de OpenAI
    a partir del perfil (seller_size, performance_level, etc.).
    """
    prompt = build_prompt_for_seller(row)

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # o el modelo que te hayan indicado
            messages=[
                {
                    "role": "system",
                    "content": (
                        "Eres un analista comercial senior de Mercado Libre. "
                        "Tu tarea es diseñar estrategias comerciales claras, accionables "
                        "y alineadas a objetivos de negocio."
                    ),
                },
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
            temperature=0.4,
            max_tokens=400,
        )

        return response.choices[0].message.content

    except Exception as e:
        # Fallback sencillo para no romper el notebook
        return f"[ERROR al llamar a la API]: {e}"

In [46]:
def generate_strategy_dummy(row: pd.Series) -> str:
    """
    Versión dummy: retorna el prompt que se enviaría al LLM.
    En producción, aquí se llamaría a la API del modelo generativo.
    """
    prompt = generate_strategy(row)
    return  prompt

In [48]:
row = seller_profile.iloc[4]
estrategia = generate_strategy_dummy(row)
print(estrategia)

1) Objetivo principal: El objetivo principal para el seller 000e27cea2 es mejorar su desempeño comercial y aumentar sus ventas en Mercado Libre. Para lograrlo, es fundamental implementar una estrategia comercial básica que se enfoque en optimizar su oferta de productos, ajustar precios y promociones, y mejorar la calidad de su servicio logístico.

2) Acciones concretas:
- Revisar el catálogo de productos y eliminar aquellos con baja demanda o poca rotación. Añadir nuevos productos que sean tendencia en la plataforma.
- Realizar un análisis competitivo para ajustar los precios de los productos y ofrecer promociones atractivas que destaquen frente a la competencia.
- Implementar mejoras en el servicio logístico, como reducir los tiempos de entrega, mejorar la comunicación con los compradores y garantizar la calidad en el empaquetado de los productos.

3) KPIs clave:
- Tasa de conversión: Medir la proporción de visitantes que realizan una compra en la tienda del seller, lo que indicará la